In [7]:
from bs4 import BeautifulSoup
import requests
import warnings
import time
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [8]:
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                              options=chrome_options)
    return driver

In [9]:
# 교보 베스트셀러 크롤링 함수

# 1. 통합된 Selector 정의
OL_CONTAINER_BASE_SELECTOR = 'body > div.relative.min-w-\[1440px\] > main > section > div > div > section > ol.grid.grid-cols-1'
BOOK_ITEMS_SELECTOR = OL_CONTAINER_BASE_SELECTOR + ' > li'
WAITING_SELECTOR = OL_CONTAINER_BASE_SELECTOR + ' > li:nth-child(1)' 


def crawl_kyobo(driver):
    """
    교보문고 일간 베스트셀러 50위까지 단일 페이지에서 크롤링합니다.
    (10위 이후 구조 변경을 반영한 통합 Selector 사용)
    """
    all_data = []
    
    # 1. 단일 URL 사용 (page=1&per=50)
    url = 'https://store.kyobobook.co.kr/bestseller/online/daily?page=1&per=50'
    driver.get(url)
    print("🔍 50개 항목 로딩 중...")

    # 명시적 대기 (통합 Selector 사용)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, WAITING_SELECTOR))
        )
        print("✅ 명시적 대기 성공! 데이터 추출을 시작합니다.")
    except TimeoutException:
        print("❌ 로딩 시간 초과: 목록 컨테이너를 찾을 수 없습니다. Selector를 다시 확인하세요.")
        return pd.DataFrame()
    
    # 2. HTML 파싱 및 전체 목록 선택
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    book_items = soup.select(BOOK_ITEMS_SELECTOR)
    
    print(f"✅ 총 {len(book_items)}개의 도서 항목 감지.") # 이 값이 50이 되어야 합니다.

    # 3. 데이터 추출 (내부 Selector는 11위 기준으로 이미 정확하므로 그대로 사용)
    for i, item in enumerate(book_items, 1):
        try:
            # 1. 순위: .rank_num
            rank_tag = item.select_one('.rank_num') 
            rank = rank_tag.text.strip() if rank_tag else str(i)
            
            # 2. 제목 (11위 Selector 기반)
            title_tag = item.select_one('div.ml-4.w-full.min-w-\[516px\] > a')
            title = title_tag.text.strip()

            # 3. 저자, 출판사, 출간일 (11위 Selector 기반)
            meta_div_selector = 'div.ml-4.w-full.min-w-\[516px\] > div.line-clamp-2.flex.overflow-hidden.whitespace-normal.break-all.text-gray-800.fz-14.mt-1'
            meta_text = item.select_one(meta_div_selector).text.strip()
            
            meta_parts = meta_text.split('·') 
            author = meta_parts[0].strip() if len(meta_parts) > 0 else 'N/A'
            publisher = meta_parts[1].strip() if len(meta_parts) > 1 else 'N/A'
            publish_date = meta_parts[2].strip() if len(meta_parts) > 2 else 'N/A'
            
            # 4. 가격 (11위 Selector 기반)
            price_tag = item.select_one('div.flex.flex-col.mt-3 > div > span:nth-child(2) > span.font-bold')
            price_full = price_tag.text.strip()
            price = int(price_full.replace(',', ''))

            # 5. 할인율 추출 
            # LI 기준으로 상대화: div.flex.flex-col.mt-3 > div > span.font-bold.inline-block...
            discount_tag = item.select_one('div.flex.flex-col.mt-3 > div > span.font-bold.inline-block.align-top.text-green-800.fz-16')
            discount_rate_str = discount_tag.text.strip() if discount_tag else '0%' # 요소가 없으면 0%로 간주
            # 데이터 정제: '%' 제거 및 float으로 변환 (분석 용이성)
            discount_rate = discount_rate_str.replace('%', '')
            
            # 6. 평점 (11위 Selector 기반)
            rating_tag = item.select_one('div.flex.w-full.flex-wrap.gap-3 span.font-bold.text-black')
            rating = rating_tag.text.strip() if rating_tag else 'N/A'

            all_data.append({
                'rank': rank, 'title': title, 'author': author, 'publisher': publisher, 'publish_date': publish_date, 
                'price': price, 'discount': discount_rate, 'review_cnt': rating, 'site': '교보문고'
            })
        except (AttributeError, ValueError) as e:
            continue
            
    return pd.DataFrame(all_data)

In [10]:
# 데이터 출력

# --- 데이터 수집 및 확인 코드 ---
driver = set_chrome_driver()

# 1. 교보문고 크롤링 함수 실행
kyobo_df = crawl_kyobo(driver)

# 2. 콘솔에 수집된 데이터(DataFrame) 출력
print("\n--- 📊 수집된 교보문고 베스트셀러 데이터 ---")
print(kyobo_df) # DataFrame 내용을 콘솔에 출력합니다.
print("------------------------------------------")

# 3. 드라이버 종료
driver.quit()

# 4. 최종 결과 출력
print("교보문고 크롤링 완료. 수집 데이터 개수:", len(kyobo_df))

🔍 50개 항목 로딩 중...
✅ 명시적 대기 성공! 데이터 추출을 시작합니다.
✅ 총 50개의 도서 항목 감지.

--- 📊 수집된 교보문고 베스트셀러 데이터 ---
   rank                                         title               author  \
0     1                                      최소한의 삼국지                  최태성   
1     2                                      나나 올리브에게                   루리   
2     3                                  트렌드 코리아 2026                김난도 외   
3     4                                 절대 실패 없는 금 투자                  배재한   
4     5                                            절창                  구병모   
5     6                                       마일리지 아워                  최유나   
6     7                                  피지컬 AI 패권 전쟁                  박종성   
7     8                                  어른의 행복은 조용하다                   태수   
8     9                                    예약판매피크 코리아                  백우열   
9    10                                        어른의 품위                  최서영   
10   11                                  박곰희 연금 

In [11]:
# excel 저장 함수

def save_data_to_excel(df, sheet_name, filename):
    """
    DataFrame을 지정된 Excel 파일의 시트에 저장합니다.
    파일이 존재하면 새로운 시트를 추가하고, 없으면 새로 생성합니다.
    """
    try:
        # ExcelWriter를 사용하여 파일 열기 (mode='a'는 파일이 있으면 추가, 없으면 생성)
        # 단, Pandas의 to_excel을 ExcelWriter 없이 사용하거나,
        # with pd.ExcelWriter를 사용해 여러 시트를 작성하는 것이 일반적입니다.
        
        # 여기서는 단일 파일에 '교보문고' 시트를 저장합니다.
        # 나중에 YES24 데이터를 추가할 때 이 writer 객체를 확장하면 됩니다.
        
        # with 블록을 사용하여 파일 저장 후 자동으로 닫히도록 처리
        with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            
        print(f"✅ 데이터가 '{filename}' 파일의 '{sheet_name}' 시트에 성공적으로 저장되었습니다.")

    except Exception as e:
        print(f"❌ Excel 파일 저장 중 오류 발생: {e}")

In [16]:
# --- 메인 실행 로직 ---

# 1. 파일명 생성 (중복 방지를 위해 현재 시각 포함)
# EXCEL_FILENAME = 'kyobo_best_' + time.strftime("%Y%m%d_%H%M") + '.xlsx'
EXCEL_FILENAME = 'rank_kyobo' + '.xlsx'

# 2. 크롤링 실행 (driver 정의와 quit()은 이미 진행했다고 가정)
# driver = set_chrome_driver()
# kyobo_df = crawl_kyobo(driver)
# driver.quit()

# 💡 이미 주피터 환경에서 kyobo_df가 메모리에 저장되어 있으므로 바로 저장 함수 호출
if 'kyobo_df' in locals() and not kyobo_df.empty:
    save_data_to_excel(kyobo_df, '교보문고', EXCEL_FILENAME)
else:
    print("❌ kyobo_df에 저장된 데이터가 없거나 비어 있어 Excel 저장을 건너뜁니다.")

✅ 데이터가 'rank_kyobo.xlsx' 파일의 '교보문고' 시트에 성공적으로 저장되었습니다.


In [15]:
%pip install xlsxwriter

     -------------------------------------- 175.3/175.3 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
